# Diagnostyka raka skóry z wykorzystaniem analizy obrazów dermatologicznych - trening modeli hybrydowych

Celem tego treningu jest przygotowanie i wytrenowanie modeli hybrydowych, czyli łączących sieć neuronową z jednym z klasycznych modeli uczenia maszynowego. Wykorzystamy tutaj wyczone modele DenseNet-121 z pliku `02_trening_densenet.ipynb`, które wykorzystamy jako ekstraktory cech (odetniemy warstwę klasyfikacyjną). Wszelki cechy wyprodukowane przez te ekstraktory będą następnie wprowadzane do jednego z dwóch modeli: maszyny wektorów nośnych i lasy losowego. Modele te będą podejmowały decyzję diagnostyczną.

## Konfiguracja środowiska

Importujemy potrzebne biblioteki.

In [ ]:
import os
import shutil
import glob
from tqdm import tqdm
from google.colab import drive, files
import kagglehub
import random
import time
import copy
import json
import joblib
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, loguniform
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

Definiujemy zmienne globalne.

In [ ]:
# parametry globalne
SEED = 42
BATCH_SIZE = 64                                                         # wielkość partii danych
NUM_WORKERS = min(4, multiprocessing.cpu_count())                       # liczba wątków procesora
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 8                                                         # liczba klas
MODEL_NAME = 'DenseNet-121'                                             # nazwa modelu

# parametry transformacji obrazów
IMG_SIZE = 300                      # wymiar wejściowy obrazu
NORM_MEAN = [0.485, 0.456, 0.406]   # wartości średniej do normalizacji
NORM_STD = [0.229, 0.224, 0.225]    # wartości odchyleń do normalizacji

# parametry czyszczenia danych
VIGNETTE_THRESHOLD = 20      # próg czerni
EXTRA_CROP = 0.05            # margines dodatkowego przycięcia (5%)
MIN_FILL_RATIO = 0.95        # bezpiecznik dla zdjęć pełnokadrowych

# podział danych
CV_FOLDS = 5                 # liczba foldów walidacyjnych

Aby móc uzyskać dostęp do GPU w notatniku należy wybrać:

$\textbf{Środowisko wykonawcze} ⟶ \textbf{Zmień typ środowiska wykonwaczego} ⟶ \text{Wybieramy jedno z dostępnych GPU} ⟶ \textbf{Zapisz}$

Po takiej sekwencji operacji należy przepuścić kod notatnika od samego początku, gdyż zostanie przydzielona nowa sesja, a poprzednia zostanie usunięta.

Ustawiamy ziarno losości dla powtarzalności wyników.

In [ ]:
def set_seed(seed=42):
    """Ustawia ziarno losowości dla reprodukowalności wyników."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
set_seed(SEED)

Dodatkowo konfigurujemy wygląd wykresów w całym notatniku.

In [ ]:
# bazowy styl seaborn z siatką
sns.set_theme(style="whitegrid", context="notebook", font_scale=1.1)

# ustawienia matplotlib
plt.rcParams.update({
    'figure.figsize': (10, 6),       # domyślny rozmiar wykresu
    'figure.dpi': 120,               # wyraźny wygląd wykresów w Colabie
    'savefig.dpi': 300,              # wysoka rozdzielczość zapisywanych wykresów
    'savefig.bbox': 'tight',         # automatycznie przycinanie białych marginesów przy zapisie
    'font.family': 'sans-serif',     # wykorzystanie czcionki bezszerfyowej
    'font.sans-serif': ['DejaVu Sans', 'Arial'],
    'axes.spines.top': False,        # usuwanie górnej ramki
    'axes.spines.right': False,      # usuwanie prawej ramki
})

# wyostrzone grafiki w Colabie
%config InlineBackend.figure_format = 'retina'

Konfigurujemy wszelki potrzebne ścieżki.

In [ ]:
# montowanie Google Drive
drive.mount('/content/drive')

# podajemy nazwę głównego folderu projektu
PROJECT_FOLDER_NAME = 'Implementacja'
BASE_DIR = os.path.join('/content/drive/MyDrive/Diagnostyka raka skóry z wykorzystaniem analizy obrazów dermatologicznych', PROJECT_FOLDER_NAME)

# podajemy ścieżkę w Colabie do zapisania pobieranych danych
DATA_ROOT_DIR = '/content/ISIC2019'
IMG_DIR = os.path.join(DATA_ROOT_DIR, 'images')
PROCESSED_DIR = os.path.join(DATA_ROOT_DIR, 'images_processed')

# definijemy podfoldery
MODELS_DIR = os.path.join(BASE_DIR, 'Modele')
RESULTS_DIR = os.path.join(BASE_DIR, 'Wyniki')
DATA_DIR = os.path.join(BASE_DIR, 'Dane')

# tworzymy foldery na Google Drive (jeśli nie istnieją)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Katalog roboczy projektu: {BASE_DIR}")
print(f"Dane będą zapisywane w: {DATA_ROOT_DIR}")

### Konfiguracja Kaggle API

Kaggle API jest nam potrzebne do pobrania danych bezpośrednio do notatnika. Folder ZIP jest bardzo obszernym plikiem i ręczne wgrywanie go na Google Drive może zająć bardzo dużo czasu.

Kofigurujemy ścieżkę dla Kaggle API. Klucz będzie zapisywany bezpośrednio w folderze projektu.

In [ ]:
kaggle_key_drive_path = os.path.join(BASE_DIR, 'kaggle.json')

Tworzymy w Colabie folder na plik z Kaggle API.

In [ ]:
kaggle_sys_dir = '/root/.kaggle'
os.makedirs(kaggle_sys_dir, exist_ok=True)

Wprowadzamy Kaggle API. Klucz ten należy utworzyć indywidualnie dla każdego użytkownika notatników z implementacją pracy inżynierskie. Aby go zdobyć należy:
1. Zalogować się na swój profil na stronie Kaggle.
2. Wejście w ustawienia swojego profilu (kilkamy na swoje zdjęcie profilowe i wybieramy ,,Settings'').
3. W sekcji ,,API'' klikamy ,,Generate New Token'', następnie wprowadzamy nazwę tokenu i klikamy ,,Generate''.
4. Kopiujemy kod z okienka ,,API TOKEN''.
5. W dowolnym edytorze tekstu (np. Notatniku) wprowadzamy tekst o następującej strykturze
$$\{\text{"username":"nazwa_uzytkownika_kaggle","key":"skopiowane_kaggle_api"}\}$$
i zapisujemy plik jako ,,kaggle.json''.
W poniższej funkcji zostaniemy poproszeni o wgranie utworzonego pliku.


In [ ]:
# sprawdzamy czy klucz jest już na Drive
if not os.path.exists(kaggle_key_drive_path):
    print(f"UWAGA: Nie znaleziono pliku kaggle.json w lokalizacji: {kaggle_key_drive_path}")
    print("Proszę przesłać plik kaggle.json teraz (zostanie zapisany na Drive):")

    # przesyłamy plik kaggle.json
    uploaded = files.upload()

    for filename in uploaded.keys():
        if filename == 'kaggle.json':
            # zapisujemy trwale na Google Drive
            with open(kaggle_key_drive_path, 'wb') as f:
                f.write(uploaded[filename])
            print(f"Zapisano kaggle.json w: {kaggle_key_drive_path}")
        else:
            print(f"Pominięto plik: {filename} (oczekiwano kaggle.json)")

## Pobranie danych

Pobieramy, rozpakowujemy i porządkujemy zbiór ISIC2019. Nie pobieramy obrazów na dysk Google Drive, lecz do pamięci Google Colab. Pozwoli to na sprawniejsze wczytywanie plików w notatniku.

In [ ]:
if os.path.exists(kaggle_key_drive_path):
    # instalacja klucza w systemie
    shutil.copy(kaggle_key_drive_path, os.path.join(kaggle_sys_dir, 'kaggle.json'))
    os.chmod(os.path.join(kaggle_sys_dir, 'kaggle.json'), 600)
    print("Klucz Kaggle API skonfigurowany.")

    # POBIERANIE I PORZĄDKOWANIE DANYCH

    # sprawdzamy, czy folder ze zdjęciami już istnieje i jest pełny
    # (np. czy nie uruchamiamy komórki drugi raz w tej samej sesji)
    if not os.path.exists(IMG_DIR):
        print("\nPobieranie danych z Kaggle...")
        # pobieramy do folderu tymczasowego
        path = kagglehub.dataset_download("andrewmvd/isic-2019")
        print(f"Pobrano dane do folderu tymczasowego: {path}")

        # tworzymy docelowe foldery na obrazy
        os.makedirs(IMG_DIR, exist_ok=True)
        os.makedirs(PROCESSED_DIR, exist_ok=True)

        # przenosimy metadane w folderu z danymi w Colabie
        for csv_file in glob.glob(os.path.join(path, "*.csv")):
            shutil.copy(csv_file, DATA_ROOT_DIR)
            print(f"Przeniesiono: {os.path.basename(csv_file)}")

        # szukamy rekursywnie zdjęć we wszystkich podfolderach pobranych danych
        jpg_files = glob.glob(os.path.join(path, "**", "*.jpg"), recursive=True)

        print(f"Znaleziono {len(jpg_files)} zdjęć.")

        for file_path in tqdm(jpg_files, desc="Kopiowanie zdjęć", unit="plik"):
            # przenosimy plik do wspólnego folderu
            shutil.copy(file_path, os.path.join(IMG_DIR, os.path.basename(file_path)))
        print("Przeniesiono zdjęcia do folderu z danymi.")

        # weryfikujemy liczbę plików
        num_images = len(os.listdir(IMG_DIR))
        print(f"\nSUKCES: Dane gotowe w {DATA_ROOT_DIR}")
        print(f"Liczba zdjęć w {IMG_DIR}: {num_images}")

    else:
        print("Dane są już pobrane i uporządkowane.")
        print(f"Lokalizacja zdjęć: {IMG_DIR}")

else:
    print("BŁĄD: Brak pliku kaggle.json. Nie można pobrać danych.")

## Przygotowanie danych do treningu

Wczytujemy metadane.

In [ ]:
metadata_processed_path = os.path.join(DATA_DIR, 'ISIC_metadata_processed.csv')
df = pd.read_csv(metadata_processed_path)
df.head()

Wczytujemy i przycinamy zdjęcia.

In [ ]:
def crop_vignette(image_path, output_path, threshold=20, extra_crop_percent=0.05, min_fill_ratio=0.95):
    """
    Funkcja usuwająca czarne ramki (winiety) z obrazu z dodatkowym marginesem bezpieczeństwa
    oraz mechanizmem chroniącym zdjęcia pełnokadrowe.

    Parametry:
    - image_path: ścieżka do pliku wejściowego.
    - output_path: ścieżka zapisu pliku wynikowego.
    - threshold: próg jasności (0-255). Piksele ciemniejsze niż ta wartość są traktowane jako tło.
    - extra_crop_percent: ułamek (np. 0.05 = 5%), o jaki dodatkowo zmniejszamy ramkę z każdej strony,
      aby usunąć postrzępione krawędzie lub pozostałości winiety w rogach.
    - min_fill_ratio: współczynnik wypełnienia (zakres 0.0 - 1.0). Określa próg decyzyjny,
      czy zdjęcie w ogóle wymaga przycinania. Jeśli wykryty obszar treści zajmuje większą część
      zdjęcia niż ta wartość (np. > 95%), algorytm uznaje, że winieta nie występuje (lub jest pomijalna)
      i rezygnuje z przycinania, aby nie uszkodzić zmian skórnych dotykających krawędzi.
    """

    # wczytujemy obraz z dysku
    img = cv2.imread(image_path)

    # jeśli plik jest uszkodzony lub nie istnieje, przerywamy
    if img is None:
        return False, None

    # przekształcamy obraz do skali szarości i wyznaczmy obszar zdjęcia
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h_img, w_img = gray.shape
    total_area = h_img * w_img

    # tworzymy maskę binarną:
    # - piksele > threshold (treść) dostają wartość 255 (biały)
    # - piksele <= threshold (tło/winieta) dostają wartość 0 (czarny)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # znajdujemy położenie treści
    # cv2.findNonZero zwraca listę współrzędnych wszystkich białych pikseli na masce
    coords = cv2.findNonZero(mask)

    # jeśli nie znaleziono żadnych jasnych punktów (zdjęcie jest całe czarne), kończymy
    if coords is None:
        return False, None

    # znajdujemy najmniejszy prostokąt (x, y, szerokość, wysokość), który obejmuje wszystkie jasne punkty
    x, y, w, h = cv2.boundingRect(coords)

    # obliczamy obszar czystego obrazu oraz współczynnik wypełnienia
    rect_area = w * h
    fill_ratio = rect_area / total_area

    # jeśli treść zajmuje prawie cały obraz (> 95%), to znaczy, że nie ma winiety,
    # albo są tylko mikro-paski, których nie warto ruszać, by nie uciąć zmiany
    if fill_ratio > min_fill_ratio:
        return False, None

    # ETAP PRZYCINANIA

    # obliczamy ile pikseli uciąć dodatkowo z każdej strony (5% szerokości/wysokości znalezionej ramki)
    margin_w = int(w * extra_crop_percent)
    margin_h = int(h * extra_crop_percent)

    # wyznaczamy nowe współrzędne ramki, zawężając ją do środka:
    # przesuwamy początek X w prawo i Y w dół
    new_x = x + margin_w
    new_y = y + margin_h

    # zmniejszamy szerokość i wysokość o marginesy z obu stron (lewo+prawo, góra+dół)
    new_w = w - (2 * margin_w)
    new_h = h - (2 * margin_h)

    # jeśli zdjęcie było bardzo małe lub margines był za duży i "zjadł" cały obraz (wymiar <= 0),
    # to cofamy się do wersji podstawowej (bez cięcia).
    if new_w <= 0 or new_h <= 0:
        new_x, new_y, new_w, new_h = x, y, w, h

    # sprawdzamy czy przycięcie jest konieczne
    # pobieramy oryginalne wymiary obrazu
    h_img, w_img = gray.shape

    # czy nowa, wyliczona ramka jest mniejsza niż oryginalny obraz?
    # jeśli tak, wykonujemy fizyczne cięcie obrazu
    if (new_w < w_img) or (new_h < h_img):
        # wycinanie fragmentu tablicy
        cropped_img = img[new_y:new_y+new_h, new_x:new_x+new_w]

        # zapisujemy wynik na dysk
        cv2.imwrite(output_path, cropped_img)

        # zwracamy sukces oraz informację, ile pikseli ucięto z każdej strony (góra, dół, lewo, prawo)
        return True, (new_y, h_img-(new_y+new_h), new_x, w_img-(new_x+new_w))

    # jeśli nie trzeba było ciąć (ramka pokrywa się z całym obrazem), zwracamy False
    return False, None

In [ ]:
# filtrujemy pliki, biorąc tylko te z rozszerzeniem .jpg lub .png
image_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith(('.jpg', '.png'))]

cropped_count = 0
copied_count = 0

print(f"Start: {len(image_files)} zdjęć. Dodatkowe cięcie: {EXTRA_CROP*100}%")

# pętla po wszystkich zdjęciach z użyciem paska postępu
for f in tqdm(image_files, desc="Przetwarzanie"):

    # tworzenie pełnych ścieżek
    src_path = os.path.join(IMG_DIR, f)
    dst_path = os.path.join(PROCESSED_DIR, f)

    # wywołujemy cięcie
    is_cropped, _ = crop_vignette(src_path, dst_path, threshold=VIGNETTE_THRESHOLD, extra_crop_percent=EXTRA_CROP, min_fill_ratio=MIN_FILL_RATIO)

    if is_cropped:
        # jeśli funkcja wykonała cięcie i zapisała plik -> zwiększ licznik
        cropped_count += 1
    else:
        # jeśli funkcja nie przycięła zdjęcia (zwróciła False) -> kopiujemy oryginał bez zmian
        shutil.copy(src_path, dst_path)
        copied_count += 1

print("\nZakończono przycinanie.")
print(f"Przycięto: {cropped_count}")
print(f"Skopiowano (bez zmian): {copied_count}")

## Budowa pętli treningowej

Przejdziemy do wytrenowania modeli hybrydowych. Zdecydowaliśmy na modele DenseNet-121 + SVM oraz DenseNet-121 + Las losowy. Będziemy tutaj traktować model sieci splotowej jak ekstraktor cech, które zostaną następnie wprowadzony do klasyfikatorów.

Przywołujemy klasę `ISICDataset`.

In [ ]:
class ISICDataset(Dataset):
    """
    Niestandardowy Dataset PyTorch do wczytywania obrazów dermatologicznych ISIC.

    Klasa wczytuje obrazy na podstawie ścieżek z DataFrame, konwertuje je do RGB
    i zwraca w formie gotowej do przetworzenia przez model.
    """

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Inicjalizuje dataset.

        Args:
            dataframe (pd.DataFrame): Ramka danych zawierająca co najmniej kolumny:
                                      - 'nazwa_pliku': nazwa pliku z rozszerzeniem (np. 'IMG_1.jpg')
                                      - 'target': numeryczna etykieta klasy (int).
            root_dir (str): Ścieżka do katalogu głównego, w którym znajdują się obrazy.
            transform (callable, optional): Opcjonalne transformacje (np. augmentacja, normalizacja)
                                            aplikowane na obrazie. Domyślnie None.
        """
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        """Zwraca całkowitą liczbę próbek w zbiorze."""
        return len(self.df)

    def __getitem__(self, idx):
        """
        Pobiera pojedynczą próbkę danych o podanym indeksie.

        Args:
            idx (int/tensor): Indeks próbki do pobrania.

        Returns:
            tuple: Krotka (image, label), gdzie:
                   - image (PIL.Image lub Tensor): Obraz po transformacjach.
                   - label (int): Numeryczna etykieta klasy.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # pobranie nazwy pliku
        img_name = self.df.iloc[idx]['nazwa_pliku']
        img_path = os.path.join(self.root_dir, img_name)

        # wczytanie obrazu i konwersja na RGB (dla bezpieczeństwa)
        image = Image.open(img_path).convert('RGB')

        # pobranie etykiety numerycznej (target)
        label = self.df.iloc[idx]['target']

        if self.transform:
            image = self.transform(image)

        return image, label

Nie bedziemy korzystać już z augumentacji danych, ponieważ modele DenseNet-121 są już wytrenowane. Wystarczą transformacje zbioru walidacyjnego stosowane do zbioru treningowego i walidacyjnego.

In [ ]:
working_size = int(IMG_SIZE * 1.25)

feature_extraction_transforms =  transforms.Compose([
    # zmiana wymiarów
    transforms.Resize(working_size),
    # wycinamy środek obrazu w docelowych wymiarach
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(NORM_MEAN, NORM_STD)
])

Tworzymy funkcję, które będzie wczytywać model i jego wytrenowane wagi, a następnie zmodyfikuje warstwę klasyfikacyjną, aby model stał się ekstraktorem cech.

In [ ]:
def get_densenet_extractor(weights_path, device):
    """
    Wczytuje wytrenowany model DenseNet-121 i konwertuje go na ekstraktor cech.

    Funkcja najpierw odtwarza architekturę użytą podczas treningu (aby poprawnie
    wczytać wagi), a następnie zamienia warstwę klasyfikacyjną na funkcję tożsamościową,
    dzięki czemu model zwraca wektory cech zamiast predykcji klas.

    Args:
        weights_path (str): Ścieżka do pliku .pth z zapisanymi wagami modelu.
        device (torch.device): Urządzenie (CPU/GPU), na które ma trafić model.

    Returns:
        torch.nn.Module: Model w trybie ewaluacji, zwracający cechy (embeddings).
                         Zwraca None, jeśli wczytanie wag się nie powiedzie.
    """
    # inicjalizujemy pusty model
    model = models.densenet121(weights=None)
    in_features = model.classifier.in_features

    # odtwarzamy warstwę klasyfikacyjną z notatnika z treningu
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(in_features, NUM_CLASSES)
    )

    # wczytujemy wagi
    try:
        state_dict = torch.load(weights_path, map_location=device)
        model.load_state_dict(state_dict)
        print(f"  -> Wczytano wagi modelu: {os.path.basename(weights_path)}")
    except FileNotFoundError:
        print(f"  [BŁĄD] Nie znaleziono pliku wag: {weights_path}")
        return None
    except Exception as e:
        print(f"  [BŁĄD] Problem z wczytaniem wag: {e}")
        return None

    # modyfikujemy warstwę klasyfikacyjną, aby zwracała ostatni wynik warstw splotowych
    model.classifier = nn.Identity()

    # przenosimy na GPU (jeśli dostępne)
    model.to(device)
    # włączamy tryb ewaluacji
    model.eval()
    return model

Defniujemy wartości hiperparametrów do przeszukiwania podczas trenowania klasyfikatorów. Ze względu na nierównomierne rozłożenie klas wprowadzamy własnej wagi do przeszukiwania hiperparametrów.

In [ ]:
# zliczamy elementy klas
class_counts = df['target'].value_counts().sort_index()
total_samples = len(df)

# wyznaczamy wagi i normalizujemy
weights_sqrt = 1. / np.sqrt(class_counts.values)
weights_sqrt = weights_sqrt / np.mean(weights_sqrt)

# tworzymy słownik {klasa: waga}
custom_weights_dict = dict(zip(class_counts.index, weights_sqrt))

# sprawdzamy słownik
for cls, w in custom_weights_dict.items():
    print(f"  Klasa {cls}: {w:.4f}")

Definiujemy rozkład wartości hiperparametró do przeszukania dla SVM.

In [ ]:
param_dist_svm = {
    'C': loguniform(1e-2, 1e2),
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None, custom_weights_dict]
}

Analogicznie definiujemy dla lasu losowego.

In [ ]:
param_dist_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'class_weight': ['balanced', 'balanced_subsample', None, custom_weights_dict]
}

Definiujemy funkcję, która będzie wczytywać model dla wprowadzonego numeru podzbioru.

In [ ]:
def load_feature_extractor(fold_idx):
    """
    Wczytuje model DenseNet (jako ekstraktor cech) dla wskazanego foldu walidacji krzyżowej.

    Funkcja konstruuje ścieżkę do pliku z wagami na podstawie indeksu foldu
    (np. 'densenet121_fold_0.pth'), a następnie inicjalizuje model w trybie ewaluacji.

    Args:
        fold_idx (int): Numer foldu (np. 0, 1, 2...), dla którego ma zostać wczytany model.

    Returns:
        torch.nn.Module: Model gotowy do ekstrakcji cech.
                         Zwraca None, jeśli plik z wagami nie istnieje (z wypisaniem ostrzeżenia).
    """
    # tworzymy ścieżkę pliku
    weights_filename = f'{MODEL_NAME}_fold{fold_idx}.pth'
    model_path = os.path.join(MODELS_DIR, weights_filename)

    # przygotowujemy model
    model = get_densenet_extractor(model_path, DEVICE)

    if model is None:
        print(f"[WARNING] Pominięto fold {fold_idx} z powodu braku pliku wag.")
        return None

    return model

Analogicznie definiujemy funkcję, która utworzy zbiór treningowy i walidacyjny oraz loadery.

In [ ]:
def get_fold_data(df, fold_idx):
    """
    Przygotowuje DataLoadery do etapu ekstrakcji cech (trening hybrydowy).

    Funkcja dzieli dane na zbiór treningowy (n-1 foldów) i walidacyjny (1 fold)
    w oparciu o indeks foldu. Stosuje transformacje deterministyczne (bez losowej augmentacji),
    co jest wymagane do wygenerowania stabilnych wektorów cech dla klasyfikatorów.

    Args:
        df (pd.DataFrame): Główna ramka danych z metadanymi (wymaga kolumn 'typ_puli' i 'nr_podzbioru').
        fold_idx (int): Indeks foldu, który ma służyć jako zbiór walidacyjny (np. 0, 1... 4).

    Returns:
        tuple: Krotka (train_loader, val_loader) skonfigurowana do sekwencyjnego przetwarzania danych
               (shuffle=False dla obu loaderów).
    """
    # filtrujemy zbiór treningowy i walidacyjny
    train_df = df[(df['typ_puli'] == 'trening') & (df['nr_podzbioru'] != fold_idx)].reset_index(drop=True)
    val_df = df[(df['typ_puli'] == 'trening') & (df['nr_podzbioru'] == fold_idx)].reset_index(drop=True)

    # konstruujemy datasety
    train_ds = ISICDataset(train_df,IMG_DIR, transform=feature_extraction_transforms)
    val_ds = ISICDataset(val_df, IMG_DIR, transform=feature_extraction_transforms)

    # konstruujemy loadery
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    return train_loader, val_loader

Konstruujemy teraz funkcje, których zadaniem będzie ekstrakcja cech za pomocą wczytanego modelu, a następnie ustandaryzowanie ich (w szczególności dla modelu SVM). Dodatkowo mierzy czas ekstrakcji cech.

In [ ]:
def extract_features(model, loader, device):
    """
    Generuje wektory cech dla całego zbioru danych przy użyciu modelu.

    Funkcja iteruje przez DataLoader, przetwarza obrazy w trybie ewaluacji (bez liczenia gradientów)
    i zwraca wynikowe cechy w formacie NumPy, gotowym dla klasycznych modeli ML.

    Args:
        model (torch.nn.Module): Model sieci neuronowej pełniący rolę ekstraktora cech.
        loader (DataLoader): DataLoader dostarczający dane (zalecane shuffle=False).
        device (torch.device): Urządzenie obliczeniowe (CPU lub CUDA).

    Returns:
        tuple: Krotka (X, y), gdzie:
               - X (np.ndarray): Macierz cech o wymiarach [liczba_próbek, liczba_cech].
               - y (np.ndarray): Wektor etykiet o wymiarach [liczba_próbek].
    """
    model.eval()  # włączamy tryb ewaluacji

    all_features = []
    all_labels = []

    # wyłączamy obliczanie gradientów dla oszczędności pamięci i przyspieszenia
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Ekstrakcja cech", leave=False):
            inputs = inputs.to(device)

            # Przepuszczamy obraz przez sieć
            features = model(inputs)

            # przenosimy wyniki z GPU na CPU i zamieniamy na numpy array
            all_features.append(features.cpu().numpy())
            all_labels.append(labels.numpy())

    # łączymy wyniki ze wszystkich batchy w jedną dużą tablicę
    # X będzie miało wymiar [liczba_zdjęć, liczba_cech]
    X = np.concatenate(all_features, axis=0)
    y = np.concatenate(all_labels, axis=0)

    return X, y

In [ ]:
def process_features(model, train_loader, val_loader, fold_idx):
    """
    Realizuje proces ekstrakcji cech, ich standaryzacji oraz zapisu modelu skalującego.

    Funkcja wykorzystuje podany model do transformacji obrazów na wektory cech.
    Następnie cechy są skalowane przy użyciu StandardScaler. Skaler jest dopasowywany (fit)
    wyłącznie do danych treningowych, a następnie aplikowany do walidacyjnych, co zapobiega
    wyciekowi informacji (data leakage). Dopasowany skaler jest zapisywany na dysku.

    Args:
        model (torch.nn.Module): Model ekstraktora cech (powinien być w trybie eval).
        train_loader (DataLoader): Loader dostarczający dane treningowe.
        val_loader (DataLoader): Loader dostarczający dane walidacyjne.
        fold_idx (int): Indeks bieżącego foldu (wykorzystywany do unikalnego nazwania
                        zapisywanego pliku ze skalerem).

    Returns:
        tuple: Krotka (X_train, y_train, X_val, y_val, extraction_time), gdzie:
            - X_train, X_val (np.ndarray): Ustandaryzowane macierze cech.
            - y_train, y_val (np.ndarray): Wektory etykiet odpowiadające cechom.
            - extraction_time (float): Czas trwania samej ekstrakcji cech (w sekundach).

    Side Effects:
        Zapisuje plik 'scaler_fold_{fold_idx}.joblib' w katalogu wskazanym przez globalną
        zmienną MODELS_DIR.
    """
    print("Rozpoczynam ekstrakcję cech.")
    start_time = time.time()

    # ekstrakcja cech na zbiorze treningowym i walidacyjnym
    X_train, y_train = extract_features(model, train_loader, DEVICE)
    X_val, y_val = extract_features(model, val_loader, DEVICE)

    extraction_time = time.time() - start_time
    print(f"Ekstrakcja zakończona w {extraction_time:.1f}s.")

    # standaryzacja cech
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    print("Dane ustandaryzowano.")

    # zapisujemy StandardScaler
    scaler_path = os.path.join(MODELS_DIR, f'scaler_fold_{fold_idx}.joblib')
    joblib.dump(scaler, scaler_path)
    print(f"Zapisano scaler do: {scaler_path}")

    return X_train, y_train, X_val, y_val, extraction_time

Zbudujemy teraz pętlę, które będzie przeprowadzać trening modeli na jednym foldzie. Do tego potrzebujemy funkcji do obliczania sepcyficzności (z `02_model_densenet.ipynb`) oraz funkcji do trenowania i dostrojenia klasyfikatora.

In [ ]:
def calculate_specificity(y_true, y_pred, num_classes):
    """
    Oblicza średnią specyficzność dla klasyfikacji wieloklasowej.

    Funkcja wykorzystuje strategię One-vs-Rest, obliczając specyficzność niezależnie
    dla każdej klasy (binarna klasyfikacja: dana klasa vs reszta), a następnie zwraca
    ich średnią arytmetyczną. Jest to kluczowa metryka w diagnostyce,
    określająca zdolność modelu do poprawnego wykluczania danej choroby.

    Args:
        y_true (array-like): Wektor rzeczywistych etykiet.
        y_pred (array-like): Wektor przewidzianych etykiet.
        num_classes (int): Całkowita liczba klas w modelu.

    Returns:
        float: Uśredniona wartość specyficzności (zakres 0.0 - 1.0).
    """
    cm = confusion_matrix(y_true, y_pred, labels=range(num_classes))
    specificity_per_class = []

    for i in range(num_classes):
        # dla klasy 'i':
        # TN = suma wszystkich próbek, które nie są 'i' i nie zostały przewidziane jako 'i'
        # FP = próbki, które nie są 'i', ale zostały przewidziane jako 'i'

        # wartości w macierzt pomyłek:
        # cm[i, i] to TP
        # cm[:, i] to kolumna predykcji (suma to TP + FP) -> FP = suma_kol - TP
        # cm[i, :] to wiersz prawdy (suma to TP + FN)
        # suma całkowita to total

        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        tn = cm.sum() - (tp + fp + fn)

        # specyficzność = TN / (TN + FP) z mechanizm uniknięcia dzielenia przez 0
        if (tn + fp) > 0:
            spec = tn / (tn + fp)
        else:
            spec = 0.0
        specificity_per_class.append(spec)

    return np.mean(specificity_per_class)

In [ ]:
def train_and_tune_classifier(name, clf_obj, param_dist, X_train, y_train):
    """
    Przeprowadza optymalizację hiperparametrów i trening końcowy klasyfikatora.

    Funkcja wykorzystuje RandomizedSearchCV (losowe przeszukiwanie przestrzeni parametrów)
    z 3-krotną walidacją krzyżową, aby znaleźć najlepszą konfigurację modelu,
    a następnie zwraca estymator wytrenowany na pełnym zbiorze treningowym.

    Args:
        name (str): Nazwa modelu (np. 'SVM'), używana do logowania postępu.
        clf_obj (sklearn.base.BaseEstimator): Instancja klasyfikatora (np. SVC, RandomForestClassifier).
        param_dist (dict): Przestrzeń hiperparametrów do przeszukania (słownik lub rozkłady).
        X_train (np.ndarray): Macierz cech treningowych.
        y_train (np.ndarray): Wektor etykiet treningowych.

    Returns:
        tuple: Krotka (best_estimator, best_params, duration), gdzie:
               - best_estimator: Najlepszy, wytrenowany model gotowy do predykcji.
               - best_params (dict): Słownik z najlepszymi znalezionymi parametrami.
               - duration (float): Czas trwania procesu strojenia (w sekundach).
    """
    print(f"\nTrening i dostrajanie {name}")
    start_time = time.time()

    # dostrajanie hiperparametrów
    search = RandomizedSearchCV(
        clf_obj,
        param_dist,
        n_iter=15,
        cv=3,
        n_jobs=-1,
        random_state=SEED,
        verbose=1
    )

    # trening najlepszego modelu
    search.fit(X_train, y_train)
    duration = time.time() - start_time

    return search.best_estimator_, search.best_params_, duration

In [ ]:
def train_and_evaluate_classifiers(X_train, y_train, X_val, y_val, fold_idx, extraction_time):
    """
    Zarządza procesem treningu, walidacji i raportowania dla klasyfikatorów klasycznych (hybrydowych).

    Funkcja iteruje przez zdefiniowaną listę modeli (SVM, Las Losowy) i dla każdego z nich:
    1. Uruchamia procedurę strojenia hiperparametrów (np. RandomizedSearchCV) i treningu.
    2. Zachowuje najlepszy znaleziony model (estymator).
    3. Wykonuje predykcję na zbiorze walidacyjnym.
    4. Oblicza zestaw metryk diagnostycznych (MCC, czułość, specyficzność, precyzja, dokładność).
    5. Agreguje wyniki, predykcje oraz obiekty modeli w struktury gotowe do zwrotu.

    Args:
        X_train (np.ndarray): Macierz cech zbioru treningowego (wyekstrahowana z CNN).
        y_train (np.ndarray): Wektor etykiet zbioru treningowego.
        X_val (np.ndarray): Macierz cech zbioru walidacyjnego.
        y_val (np.ndarray): Wektor etykiet zbioru walidacyjnego.
        fold_idx (int): Numer bieżącego foldu (używany do logowania wyników).
        extraction_time (float): Czas [s] zużyty na ekstrakcję cech (dodawany do czasu treningu klasyfikatora).

    Returns:
        tuple: Krotka (fold_results, fold_preds, trained_models), zawierająca:
            - fold_results (list): Lista słowników z obliczonymi metrykami i najlepszymi parametrami.
            - fold_preds (dict): Słownik zawierający listy etykiet prawdziwych i przewidzianych
              (struktura: {'NazwaModelu': {'y_true': [...], 'y_pred': [...]}}).
            - trained_models (dict): Słownik zawierający wytrenowane obiekty modeli (np. sklearn.svm.SVC),
              gotowe do zapisu (dump) lub dalszej inferencji.
    """

    # definiujemy modele
    classifiers = [
        ('SVM', SVC(probability=True, random_state=SEED, cache_size=2000, max_iter=3000), param_dist_svm),
        ('Las losowy', RandomForestClassifier(random_state=SEED, n_jobs=None), param_dist_rf)
    ]

    fold_results = []
    fold_preds = {'SVM': {'y_true': [], 'y_pred': []}, 'Las losowy': {'y_true': [], 'y_pred': []}}
    trained_models = {} # słownik na modele

    for name, clf_obj, param_dist in classifiers:
        # trening
        best_model, best_params, train_time = train_and_tune_classifier(
            name, clf_obj, param_dist, X_train, y_train
        )

        # zapisujemy model
        trained_models[name] = best_model

        # całkowity czas treningu (ekstrakcja + trening)
        total_time = extraction_time + train_time

        # predykcja
        preds = best_model.predict(X_val)

        # obliczanie metryk
        acc = accuracy_score(y_val, preds)
        mcc = matthews_corrcoef(y_val, preds)
        sens = recall_score(y_val, preds, average='macro', zero_division=0)
        prec = precision_score(y_val, preds, average='macro', zero_division=0)
        spec = calculate_specificity(y_val, preds, NUM_CLASSES)

        print(f"Wynik dla {name}:")
        print(f"\tMCC = {mcc:.4f}")
        print(f"\tCzułość = {sens:.4f}")
        print(f"\tDokładność = {acc:.4f}")
        print(f"\tSpecyficzność = {spec:.4f}")
        print(f"\tPrecyzja = {prec:.4f}")

        # zapis wyników
        fold_results.append({
            'fold': fold_idx,
            'model': name,
            'val_dokładność': acc,
            'val_mcc': mcc,
            'val_czułość': sens,
            'val_specyficzność': spec,
            'val_precyzja': prec,
            'czas_trwania_foldu': total_time,
            'najlepsze_parametry': best_params
        })

        fold_preds[name]['y_true'].extend(y_val.tolist())
        fold_preds[name]['y_pred'].extend(preds.tolist())

    return fold_results, fold_preds, trained_models

## Trening modeli hybrydowych

Możemy teraz przejść do przeprowadzenia pełnego treningu modeli hybrydowych. Wprowadzamy zabezpieczenia na wypadek przerwania sesji i utraty wyników.

In [ ]:
hybrid_results = []
all_hybrid_preds = {
    'SVM': {'y_true': [], 'y_pred': []},
    'Las losowy': {'y_true': [], 'y_pred': []}
}

print(f"Rozpoczynam trening hybrydowy ({CV_FOLDS} foldów)...")

for fold_idx in range(CV_FOLDS):
    # definiujemy ścieżki plików
    results_path = os.path.join(RESULTS_DIR, f'wyniki_modeli_hybrydowych_fold_{fold_idx}.json')
    preds_path = os.path.join(RESULTS_DIR, f'predykcje_modeli_hybrydowych_fold_{fold_idx}.json')

    # definiujemy ścieżki modeli
    svm_path = os.path.join(MODELS_DIR, f'svm_fold_{fold_idx}.pkl')
    rf_path = os.path.join(MODELS_DIR, f'rf_fold_{fold_idx}.pkl')

    # definiujemy ścieżkę do tymczasowego pliku z cechami
    features_cache_path = os.path.join(RESULTS_DIR, f'features_cache_fold_{fold_idx}.npz')

    # MECHANIZM WZNAWIANIA
    if os.path.exists(results_path) and os.path.exists(preds_path) and os.path.exists(svm_path) and os.path.exists(rf_path):

        print(f"\n[INFO] Fold {fold_idx} ukończony. Wczytuję wyniki.")

        # wczytujemy wyniki
        with open(results_path, 'r') as f:
            fold_res = json.load(f)
            hybrid_results.extend(fold_res) # dodajemy do głównej listy

        # wczytujemy predykcje
        with open(preds_path, 'r') as f:
            fold_preds_loaded = json.load(f)
            for model_name in ['SVM', 'Las losowy']:
                all_hybrid_preds[model_name]['y_true'].extend(fold_preds_loaded[model_name]['y_true'])
                all_hybrid_preds[model_name]['y_pred'].extend(fold_preds_loaded[model_name]['y_pred'])

        continue # przejście do kolejnego foldu

    print(f"\n{'='*5}FOLD {fold_idx}/{CV_FOLDS - 1}{'='*5}")

    # MECHANIZM EKSTRAKCJI CECH
    if os.path.exists(features_cache_path):
        print(f"[CACHE] Znaleziono zapisane cechy. Wczytuję z: {features_cache_path}")

        # wczytanie tablic NumPy
        data = np.load(features_cache_path)
        X_train = data['X_train']
        y_train = data['y_train']
        X_val = data['X_val']
        y_val = data['y_val']
        extraction_time = float(data['extraction_time'])

    else:
        print(f"[GPU] Cache nie istnieje. Uruchamiam ekstrakcję cech.")

        # wczytujemy model
        model_extractor = load_feature_extractor(fold_idx)
        if model_extractor is None:
            print(f"BŁĄD KRYTYCZNY: Brak modelu DenseNet dla foldu {fold_idx}!")
            break

        # przygotowujemy dane
        train_loader, val_loader = get_fold_data(df, fold_idx)

        # wyciągamy cechy i standaryzujemy
        X_train, y_train, X_val, y_val, extraction_time = process_features(model_extractor, train_loader, val_loader, fold_idx)

        # czyścimy pamięć
        del model_extractor
        torch.cuda.empty_cache()

        # zapisujemy cechy do pliku tymczasowego
        print(f"[CACHE] Zapisuję cechy na dysk.")
        np.savez_compressed(
            features_cache_path,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            extraction_time=extraction_time
        )

    # trenujemy i ewaluujemy klasyfikatory
    fold_res, fold_preds_dict, trained_models_dict = train_and_evaluate_classifiers(
        X_train, y_train, X_val, y_val, fold_idx, extraction_time
    )

    # zbieramy wyniki i predykcje walidacyjne
    hybrid_results.extend(fold_res)
    for model_name in ['SVM', 'Las losowy']:
        all_hybrid_preds[model_name]['y_true'].extend(fold_preds_dict[model_name]['y_true'])
        all_hybrid_preds[model_name]['y_pred'].extend(fold_preds_dict[model_name]['y_pred'])

    # zapisujemy wyniki foldu
    with open(results_path, 'w') as f:
        json.dump(fold_res, f)

    # zapisujemy predykcje foldu
    with open(preds_path, 'w') as f:
        json.dump(fold_preds_dict, f)

    # zapisujemy parametry modelu
    joblib.dump(trained_models_dict['SVM'], svm_path)
    joblib.dump(trained_models_dict['Las losowy'], rf_path)


print(f"{'#'*5} Zakończono trening modeli hybrydowych. {'='*5}")

# zapisujemy zbiorcze wyniki
with open(os.path.join(RESULTS_DIR, 'wyniki_modeli_hybrydowych_FULL.json'), 'w') as f:
    json.dump(hybrid_results, f)

# zapisujemy zbiorcze predykcje
with open(os.path.join(RESULTS_DIR, 'predykcje_modeli_hybrydowych_FULL.json'), 'w') as f:
    json.dump(all_hybrid_preds, f)

Zobaczmy na wyniki treningu. Będą to średnie wyniki z wszystkich podzbiorów.

In [ ]:
results_df = pd.DataFrame(hybrid_results)
summary = results_df.groupby('model')[['val_dokładność', 'val_mcc', 'val_czułość', 'val_specyficzność', 'val_precyzja', 'czas_trwania_foldu']].mean()

summary

## Wizualizacja wyników treningu

Spróbujemy zwizualizować wyniki treningu. W tym przypadku nie narysujemy klasycznych krzywych uczenia, ale za to możemy przyglądnąć się macierzy pomyłek i wartość czułości i specyficzności.

In [ ]:
def get_class_names_from_df(df):
    """
    Pobiera listę nazw klas posortowaną zgodnie z ich numerycznymi identyfikatorami.

    Funkcja tworzy mapowanie między kolumną 'target' (0, 1, 2...) a 'typ_zmiany' (nazwa choroby),
    gwarantując, że kolejność nazw na liście odpowiada kolejności wyjść modelu.
    Jest to niezbędne do poprawnego opisywania osi macierzy pomyłek.

    Args:
        df (pd.DataFrame): Ramka danych zawierająca kolumny 'target' (int) i 'typ_zmiany' (str).

    Returns:
        list: Lista nazw chorób (str), gdzie indeks elementu odpowiada wartości targetu (np. list[0] to nazwa dla target=0).
    """
    # sortujemy po target, żeby indeks 0 listy odpowiadał target=0
    sorted_classes = df[['target', 'typ_zmiany']].drop_duplicates().sort_values('target')
    return sorted_classes['typ_zmiany'].tolist()

In [ ]:
def plot_validation_diagnosis(all_preds_dict, hybrid_results_list, df, output_dir):
    """
    Generuje i zapisuje graficzne podsumowanie wyników walidacji krzyżowej.

    Funkcja tworzy dwa rodzaje wykresów diagnostycznych:
    1. Znormalizowane macierze pomyłek - zsumowane
       ze wszystkich foldów, pozwalające ocenić ogólną skuteczność diagnozy per klasa.
    2. Wykres rozrzutu Czułość vs Specyficzność - obrazujący stabilność
       modeli pomiędzy foldami oraz ich średni punkt pracy (kompromis).

    Args:
        all_preds_dict (dict): Słownik zawierający listy 'y_true' i 'y_pred' dla każdego modelu.
        hybrid_results_list (list): Lista słowników z metrykami dla każdego foldu.
        df (pd.DataFrame): Główna ramka danych (służy do automatycznego pobrania nazw klas).
        output_dir (str): Ścieżka do katalogu, w którym zostaną zapisane pliki PNG.
    """
    # ustalenie nazw klas
    try:
        class_names = get_class_names_from_df(df)
        print(f"Automatycznie wykryto klasy: {class_names}")
    except KeyError:
        # gdyby nazwy kolumn były inne
        print("[Ostrzeżenie] Nie znaleziono kolumny 'diagnosis'. Używam domyślnych.")
        class_names = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

    models = list(all_preds_dict.keys())

    # rysujemy macierze pomyłek
    fig, axes = plt.subplots(1, len(models), figsize=(8 * len(models), 7))
    if len(models) == 1: axes = [axes] # Obsługa przypadku jednego modelu

    for idx, model_name in enumerate(models):
        y_true = all_preds_dict[model_name]['y_true']
        y_pred = all_preds_dict[model_name]['y_pred']

        # obliczamy macierz
        cm = confusion_matrix(y_true, y_pred)
        # normalizujemy (żeby widzieć skuteczność w %)
        with np.errstate(divide='ignore', invalid='ignore'):
            cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            cm_norm = np.nan_to_num(cm_norm) # Zamienia NaN na 0

        sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names,
                    ax=axes[idx], cbar=False, annot_kws={"size": 9})

        axes[idx].set_title(f'{model_name}\nMacierz pomyłek (zsumowana z 5 foldów)', fontsize=12)
        axes[idx].set_ylabel('Prawdziwa diagnoza')
        axes[idx].set_xlabel('Przewidziana diagnoza')
        axes[idx].tick_params(axis='x', rotation=45)

    plt.tight_layout()

    # zapisujemy wykres
    save_cm_path = os.path.join(output_dir, 'walidacja_macierze_pomylek.png')
    plt.savefig(save_cm_path)
    print(f"Zapisano macierze pomyłek: {save_cm_path}")
    plt.show()

    # wykres rozrzutu: Czułość vs Specyficzność
    results_df = pd.DataFrame(hybrid_results_list)

    plt.figure(figsize=(10, 7))

    # rysujemy punkty
    sns.scatterplot(
        data=results_df,
        x='val_specyficzność',
        y='val_czułość',
        hue='model',
        style='model',
        s=150,
        palette='viridis',
        alpha=0.8
    )

    # rysujemy średnie jako większe punkty (krzyżyki)
    means = results_df.groupby('model')[['val_specyficzność', 'val_czułość']].mean()
    for name, row in means.iterrows():
        plt.scatter(row['val_specyficzność'], row['val_czułość'], marker='+', s=400, color='red', linewidth=3, label=f'Średnia {name}')

    # linie pomocnicze
    plt.axhline(0.5, color='gray', linestyle='--', alpha=0.3)
    plt.axvline(0.8, color='gray', linestyle='--', alpha=0.3)
    plt.title('Czułość vs Specyficzność', fontsize=14)
    plt.xlabel('Specyficzność', fontsize=12)
    plt.ylabel('Czułość', fontsize=12)

    # legenda
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)

    # zapisujemy wykres
    save_sc_path = os.path.join(output_dir, 'walidacja_scatter_sens_spec.png')
    plt.savefig(save_sc_path, dpi=300, bbox_inches='tight')
    print(f"Zapisano wykres scatter: {save_sc_path}")
    plt.show()

In [ ]:
plot_validation_diagnosis(all_hybrid_preds, hybrid_results, df, RESULTS_DIR)